In [1]:
import numpy as np
import pandas as pd
from msmarco.index import indexed
from msmarco.evaluate import grade_results, judge_queries
from time import perf_counter
from searcharray.similarity import bm25_similarity, classic_similarity, default_bm25

NUM_QUERIES = 10

/var/folders/c3/6754bm4d6cv9n0k3v04lx53h0000gq/T/ipykernel_56044/4126905919.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Will take a min or two to load, and a good hour to build an index first time
msmarco = indexed()

In [3]:
def or_query_search(corpus, fields, query, query_id=None, n=10, 
                    similarities=default_bm25):
    if not isinstance(fields, list):
        fields = [fields]
    if isinstance(similarities, list):
        assert len(similarities) == len(fields)
    if not isinstance(similarities, list):
        similarities = [similarities] * len(fields)
    start = perf_counter()
    scored = np.zeros(len(corpus))
    for field, similarity in zip(fields, similarities):
        boost = 1.0
        if '^' in field:
            field, boost = field.split('^')
            boost = float(boost)
        tokenizer = corpus[field].array.tokenizer
        tokens = tokenizer(query)
        scored += (np.sum([corpus[field].array.score(token, similarity=similarity) for token in tokens], axis=0) * boost)
    took = perf_counter() - start
    
    top_n_idx = np.argsort(scored)[::-1][:n]
    scores = scored[top_n_idx]
    top_n_idx, scores

    top_n = corpus[['title', 'msmarco_id']].iloc[top_n_idx]
    top_n['scores'] = scores
    top_n['query'] = query
    top_n['query_id'] = query_id
    top_n['took'] = took
    top_n['rank'] = np.arange(n) + 1
    return top_n

or_query_search(corpus=msmarco, fields="title_idx", query="cheese inside", n=100)

,title,msmarco_id,scores,query,query_id,took,rank
1622353,Cheese & Cheese Products: Cheese,D2146882,5.249580,cheese inside,None,0.034558,1
1409917,Inside,D3479997,4.808581,cheese inside,None,0.034558,2
171414,Insider,D932824,4.808581,cheese inside,None,0.034558,3
3021605,inside,D1820840,4.808581,cheese inside,None,0.034558,4
940283,inside,D1820838,4.808581,cheese inside,None,0.034558,5
...,...,...,...,...,...,...,...
1081616,Emmental Cheese,D1882782,4.248903,cheese inside,None,0.034558,96
48595,Packaged Cheese,D1277912,4.248903,cheese inside,None,0.034558,97
459143,Cheese Biscuit,D2798187,4.248903,cheese inside,None,0.034558,98
2472563,Cheese Triangles,D227158,4.248903,cheese inside,None,0.034558,99


In [4]:
from msmarco.evaluate import grade_results, judgments

msmarco_judgments = judgments().sample(NUM_QUERIES)

In [5]:

def run_all(corpus, judgments, fields, similarities=default_bm25, n=10):
    results = []
    for idx, row in judgments.iterrows():
        top_n = or_query_search(corpus=msmarco, fields=fields, similarities=similarities, n=n,
                                query=row['query'], query_id=row['query_id'])
        results.append(top_n)
    results = pd.concat(results)
    graded = grade_results(msmarco_judgments, results)
    return graded

## Run random search

Try different parameters w/ random search

In [ ]:
from random import uniform
# 0.3164444444444445 ['title_idx^1.7072861134255923', 'body_idx^9.467854536052052'] title: b,k1: 0.9054699170098017,1.3007711184048492 body: b,k1: 1.2957539903585236,1.8407221594355443

results = []

for _ in range(0, 100):
    title_boost = uniform(0, 10)
    body_boost = uniform(0, 10)
    b_title = uniform(0,2)
    k1_title = uniform(0.1,2)
    b_body = uniform(0,2)
    k1_body = uniform(0.1,2)
    
    bm25_similarity_title = bm25_similarity(b=b_title,
                                            k1=k1_title)
    bm25_similarity_body = bm25_similarity(b=b_body,
                                           k1=k1_body)
    fields = [f"title_idx^{title_boost}", f"body_idx^{body_boost}"]
    similarities = [bm25_similarity_title, bm25_similarity_body]

    graded = run_all(msmarco, msmarco_judgments,
                     fields=fields,
                     n=100,
                     similarities=default_bm25)

    queries_judged = judge_queries(graded)
    mrr = queries_judged.sum() / NUM_QUERIES
    results.append({"mrr100": mrr, "title_boost": title_boost, "body_boost": body_boost,
                    "b_title": b_title, "k1_title": k1_title,
                    "b_body": b_body, "k1_body": k1_body})
    print(mrr, fields, f"title: b,k1: {b_title},{k1_title}", f"body: b,k1: {b_body},{k1_body}")


/Users/douglas.turnbull/ws/msmarco/msmarco/evaluate.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labeled_results["grade"].fillna(0, inplace=True)


0.3312405339494289 ['title_idx^6.545695436701294', 'body_idx^8.64142240478934'] title: b,k1: 0.2907422716201624,0.41498822130808677 body: b,k1: 0.38900339222104985,1.0629352225295006


/Users/douglas.turnbull/ws/msmarco/msmarco/evaluate.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labeled_results["grade"].fillna(0, inplace=True)


0.3308039215686275 ['title_idx^3.6250354444295296', 'body_idx^4.055365895323505'] title: b,k1: 0.02111725329252523,1.0263423236318738 body: b,k1: 0.8991045649113643,0.37666838624665844


/Users/douglas.turnbull/ws/msmarco/msmarco/evaluate.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labeled_results["grade"].fillna(0, inplace=True)


...

0.34608695652173915 ['title_idx^2.1175093468038075', 'body_idx^6.4241217982818375'] title: b,k1: 1.7855068181587561,0.4809303489401823 body: b,k1: 0.7794012312565826,0.6446399140978787


/Users/douglas.turnbull/ws/msmarco/msmarco/evaluate.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labeled_results["grade"].fillna(0, inplace=True)


0.3468686868686869 ['title_idx^1.490622831385191', 'body_idx^5.6323638415957324'] title: b,k1: 0.03553352726639236,1.393999734971078 body: b,k1: 1.9055727457052984,0.30792936986520225


/Users/douglas.turnbull/ws/msmarco/msmarco/evaluate.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labeled_results["grade"].fillna(0, inplace=True)


0.34399193548387097 ['title_idx^5.560387433112731', 'body_idx^9.86931729524027'] title: b,k1: 0.695554282252286,1.0087421037180035 body: b,k1: 1.0991639878585953,0.241151636319067


In [ ]:
graded